In [1]:
from typing import TypedDict, Annotated, List
from langchain_groq import ChatGroq
from langchain_community.tools import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.graph import StateGraph, START, END, add_messages
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import ToolNode

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY")

In [3]:
memory = MemorySaver()

In [4]:
search_tool = TavilySearchResults(max_results=2)
tools = [search_tool]

/tmp/ipykernel_36658/3426219496.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  search_tool = TavilySearchResults(max_results=2)


In [5]:
llm = ChatGroq(model_name="llama-3.3-70b-versatile")

llm_with_tools = llm.bind_tools(tools=tools)

In [6]:
class AgentState(TypedDict):
    messages: Annotated[List, add_messages]

In [7]:
def model(state: AgentState):
    return {
        "messages": [llm_with_tools.invoke(state["messages"])]
    }

In [8]:
def tools_router(state: AgentState):
    last_message = state["messages"][-1]
    
    if (hasattr(last_message, "tool_calls") and len(last_message.tool_calls) > 0):
        return "tools"
    else:
        return END

In [9]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node(model, "model")
graph_builder.add_node("tools", ToolNode(tools=tools))

graph_builder.set_entry_point("model")

graph_builder.add_conditional_edges("model", tools_router)
graph_builder.add_edge("tools", "model")

graph = graph_builder.compile(checkpointer=memory, interrupt_before=["tools"])

In [11]:
config = {
    "configurable": {
        "thread_id": 1
    }
}


events = graph.stream(
    {
        "messages": [HumanMessage(content="What is the current weather in Chennai?")]
    },
    config=config,
    stream_mode="values"
)

for event in events:
    event["messages"][-1].pretty_print()

================================ Human Message =================================

What is the current weather in Chennai?
================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (7etwpywjx)
 Call ID: 7etwpywjx
  Args:
    query: current weather in Chennai


In [12]:
snapshot = graph.get_state(config=config)

snapshot.next

('tools',)

In [13]:
events = graph.stream(None, config=config, stream_mode="values")

for event in events:
    event["messages"][-1].pretty_print()

================================== Ai Message ==================================
Tool Calls:
  tavily_search_results_json (7etwpywjx)
 Call ID: 7etwpywjx
  Args:
    query: current weather in Chennai
================================= Tool Message =================================
Name: tavily_search_results_json

[{"title": "Chennai weather in September 2025 - Weather25.com", "url": "https://www.weather25.com/asia/india/tamil-nadu/chennai?page=month&month=September", "content": "weather25.com\nSearch\nweather in India\nRemove from your favorite locations\nAdd to my locations\nShare\nweather in India\n\n# Chennai weather in September 2025\n\nPatchy rain possible\nPatchy light rain\nLight rain shower\nPatchy rain possible\nPatchy rain possible\nPartly cloudy\nPatchy rain possible\nPartly cloudy\nPatchy rain possible\nPatchy rain possible\nLight rain shower\nPatchy rain possible\nCloudy\nPartly cloudy\n\n## The average weather in Chennai in September [...] | 28 Patchy rain possible 32° /2